# Conexión con la base de datos

In [1]:
import mysql.connector
import pandas as pd
from mysql.connector import Error
connection = mysql.connector.connect(host='localhost',
                                         database='users',
                                         user='root',
                                         password='C4rl1t0s')
df = pd.read_sql_query('SELECT * FROM users.clientes; LIMIT 10',connection)
df.head()

,nombre,pass
0,Chami,pienso123
1,Claudio,pass
2,Waterfoobber,123456


# Métodos compresión

Midi to Wav

In [2]:
import midi2audio
from midi2audio import FluidSynth

def midi_to_wav(midi_file):
    output_wav = midi_file.replace(".mid", ".wav")
    FluidSynth().midi_to_audio(midi_file, output_wav)
    return output_wav

Ejemplo de ejecución : 

input_midi = "nombre del archivo.mid"

output_wav = convert_midi_to_wav(input_midi)

print("Archivo WAV convertido:", output_wav)

Midi to MP3

In [3]:
import midi2audio
from midi2audio import FluidSynth
from pydub import AudioSegment
import os

def midi_to_mp3(midi_file):
    output_mp3 = midi_file.replace(".mid", ".mp3")
    # Convertir MIDI a audio WAV
    wav_file = output_mp3.replace(".mp3", ".wav")
    FluidSynth().midi_to_audio(midi_file, wav_file)

    # Convertir audio WAV a MP3
    audio = AudioSegment.from_wav(wav_file)
    audio.export(output_mp3, format="mp3")

    # Eliminar archivo WAV temporal
    os.remove(wav_file)

    return output_mp3

Ejemplo de ejecucion: 

Ruta del archivo MIDI de entrada

input_midi = "one.mid"

Llamar a la función de conversión

output_mp3 = convert_midi_to_mp3(input_midi)

print("Archivo MP3 convertido:", output_mp3)

Midi to FLAC

In [4]:
import midi2audio
from midi2audio import FluidSynth
from pydub import AudioSegment
import os

def midi_to_flac(midi_file):
    output_flac = midi_file.replace(".mid", ".flac")
    # Convertir MIDI a audio WAV
    wav_file = output_flac.replace(".flac", ".wav")
    FluidSynth().midi_to_audio(midi_file, wav_file)

    # Convertir audio WAV a FLAC
    audio = AudioSegment.from_wav(wav_file)
    audio.export(output_flac, format="flac")

    # Eliminar archivo WAV temporal
    os.remove(wav_file)

    return output_flac

Ejemplo de ejecución: 

Ruta del archivo MIDI de entrada

input_midi = "one.mid"

Llamar a la función de conversión

output_flac = convert_midi_to_flac(input_midi)

print("Archivo FLAC convertido:", output_flac)


In [ ]:
from flask import Flask, jsonify, request , send_file
from flask_cors import CORS
import mysql.connector
from flask_httpauth import HTTPBasicAuth
from midi2audio import FluidSynth
import os
from tempfile import NamedTemporaryFile  
import subprocess
import tensorflow as tf



connection = mysql.connector.connect(
    host='localhost',
    database='users',
    user='root',
    password='C4rl1t0s'
)

app = Flask(__name__)
CORS(app)  # Habilitar CORS para todas las rutas
auth = HTTPBasicAuth()

# Auth
@app.route('/login', methods=['POST'])
def login():
    data = request.get_json()
    nombre = data.get('nombre')
    password = data.get('pass')

    cursor = connection.cursor()
    query = "SELECT COUNT(*) FROM clientes WHERE nombre = %s AND pass = %s"
    cursor.execute(query, (nombre, password))
    result = cursor.fetchone()

    if result[0] > 0:
        return jsonify({'success': True})
    else:
        return jsonify({'success': False})
#Register
@app.route('/register', methods=['POST'])
def register():
    data = request.get_json()
    nombre = data.get('nombre')
    password = data.get('pass')

    cursor = connection.cursor()

    # Verificar si ya existe un elemento con el mismo nombre o contraseña
    query = "SELECT COUNT(*) FROM clientes WHERE nombre = %s OR pass = %s"
    cursor.execute(query, (nombre, password))
    result = cursor.fetchone()

    if result[0] > 0:
        # Ya existe un elemento con el mismo nombre o contraseña, devuelve un error
        return jsonify({'success': False, 'message': 'Nombre o contraseña ya existente'})

    # No existe un elemento con el mismo nombre o contraseña, agregar una nueva fila
    insert_query = "INSERT INTO clientes (nombre, pass) VALUES (%s, %s)"
    cursor.execute(insert_query, (nombre, password))
    connection.commit()

    return jsonify({'success': True, 'message': 'Elemento agregado correctamente'})

#Canciones
@app.route('/canciones', methods=['POST'])
def get_canciones():
    data = request.get_json()
    nombre = data.get('nombre')

    cursor = connection.cursor()

    # Obtener todas las canciones que coinciden con el nombre proporcionado
    query = "SELECT * FROM canciones WHERE nombre LIKE %s"
    cursor.execute(query, ('%' + nombre + '%',))
    result = cursor.fetchall()

    canciones = []
    for row in result:
        cancion = {
            'nombre': row[0],
            'titulo': row[1],
            'id': row[2],
            #'duracion': row[3]
        }
        canciones.append(cancion)

    return jsonify({'success': True},canciones)

# Ruta para manejar la inserción de canciones
@app.route('/insertar_cancion', methods=['POST'])
def insertar_cancion():
    # Obtener los datos de la canción desde la solicitud POST
    data = request.get_json()
    print(data)
    nombre = data.get('nombre')
    titulo = data.get('titulo')
    idd = data.get('id')

    # Establecer la conexión a la base de datos
    cursor = connection.cursor()

    # Ejecutar la consulta de inserción
    query = "INSERT INTO canciones (nombre, titulo, id) VALUES (%s, %s, %s)"
    cursor.execute(query, (nombre, titulo, idd))
    connection.commit()

    return jsonify({'success': True, 'message': 'Cancion agregada correctamente'})

@app.route('/convert_wav', methods=['POST'])
def convert_wav():
    if 'file' not in request.files:
        return "No se encontró el archivo MIDI", 400
    
    midi = request.files['file']
    midi.save('temp.mid')

    wav_file = midi_to_wav('temp.mid')

    # Eliminar archivo MIDI temporal
    os.remove('temp.mid')

    return send_file(wav_file, as_attachment=True), 200

@app.route('/convert_mp3', methods=['POST'])
def convert_mp3():
    if 'file' not in request.files:
        return "No se encontró el archivo MIDI", 400
    
    midi = request.files['file']
    midi.save('temp.mid')

    mp3_file = midi_to_mp3('temp.mid')

    # Eliminar archivo MIDI temporal
    os.remove('temp.mid')

    return send_file(mp3_file, as_attachment=True), 200

@app.route('/convert_flac', methods=['POST'])
def convert_flac():
    if 'file' not in request.files:
        return "No se encontró el archivo MIDI", 400
    
    midi = request.files['file']
    midi.save('temp.mid')

    flac_file = midi_to_flac('temp.mid')

    # Eliminar archivo MIDI temporal
    os.remove('temp.mid')

    return send_file(flac_file, as_attachment=True), 200

@app.route('/ejecutar-python')
def ejecutar_python():
    python_script_path = 'tensors.py'  # Ruta completa al script Python
    subprocess.run(['python', python_script_path], check=True)
    return jsonify({'message': 'Comando Python ejecutado correctamente'})
   

if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [04/Jul/2023 19:39:58] "OPTIONS /login HTTP/1.1" 200 -
127.0.0.1 - - [04/Jul/2023 19:39:58] "POST /login HTTP/1.1" 200 -
127.0.0.1 - - [04/Jul/2023 19:40:02] "POST /login HTTP/1.1" 200 -
127.0.0.1 - - [04/Jul/2023 19:51:42] "OPTIONS /insertar_cancion HTTP/1.1" 200 -


{'nombre': 'Chami', 'titulo': 'model-p-3-2023-06-21.mid', 'id': 29}


127.0.0.1 - - [04/Jul/2023 19:51:45] "POST /insertar_cancion HTTP/1.1" 200 -
127.0.0.1 - - [04/Jul/2023 19:58:10] "OPTIONS /login HTTP/1.1" 200 -
127.0.0.1 - - [04/Jul/2023 19:58:10] "POST /login HTTP/1.1" 200 -
127.0.0.1 - - [04/Jul/2023 19:58:12] "OPTIONS /canciones HTTP/1.1" 200 -
127.0.0.1 - - [04/Jul/2023 19:58:13] "POST /canciones HTTP/1.1" 200 -
127.0.0.1 - - [04/Jul/2023 19:59:00] "POST /convert_wav HTTP/1.1" 200 -
127.0.0.1 - - [04/Jul/2023 19:59:38] "POST /convert_mp3 HTTP/1.1" 200 -
127.0.0.1 - - [04/Jul/2023 20:06:33] "OPTIONS /login HTTP/1.1" 200 -
127.0.0.1 - - [04/Jul/2023 20:06:33] "POST /login HTTP/1.1" 200 -
127.0.0.1 - - [04/Jul/2023 20:06:34] "OPTIONS /canciones HTTP/1.1" 200 -
127.0.0.1 - - [04/Jul/2023 20:06:34] "POST /canciones HTTP/1.1" 200 -
127.0.0.1 - - [04/Jul/2023 20:08:09] "POST /convert_flac HTTP/1.1" 200 -
127.0.0.1 - - [04/Jul/2023 20:14:53] "OPTIONS /login HTTP/1.1" 200 -
127.0.0.1 - - [04/Jul/2023 20:14:53] "POST /login HTTP/1.1" 200 -
127.0.0.1 - - [

{'nombre': 'Claudio', 'titulo': 'model-p-3-2023-06-21.mid', 'id': 12}


127.0.0.1 - - [04/Jul/2023 20:36:17] "POST /insertar_cancion HTTP/1.1" 200 -
127.0.0.1 - - [04/Jul/2023 20:36:27] "OPTIONS /canciones HTTP/1.1" 200 -
127.0.0.1 - - [04/Jul/2023 20:36:28] "POST /canciones HTTP/1.1" 200 -
127.0.0.1 - - [04/Jul/2023 20:38:48] "POST /convert_flac HTTP/1.1" 200 -


In [ ]:
from flask import Flask
import subprocess

app = Flask(__name__)

@app.route('/ejecutar_script', methods=['GET'])
def ejecutar_script():
    try:
        subprocess.call(['python', 'tensors.py'])
        return 'El script se ha ejecutado correctamente.'
    except Exception as e:
        return f'Error al ejecutar el script: {str(e)}'

if __name__ == '__main__':
    app.run()
